In [2]:
# 📦 Imports
from pytrends.request import TrendReq
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import os

In [3]:
# Start pytrends session
pytrends = TrendReq(hl='en-US', tz=0)

In [4]:
keywords = ["meditation", "mindfulness", "breathwork", "guided meditation", "yoga nidra"]

In [5]:
# Build payload for all keywords
pytrends.build_payload(kw_list=keywords, timeframe="today 5-y", geo="")

# Fetch weekly interest over time
df_trends = pytrends.interest_over_time()

# Drop 'isPartial' column if it exists
if 'isPartial' in df_trends.columns:
    df_trends = df_trends.drop(columns='isPartial')

# Preview result
df_trends.head()

,meditation,mindfulness,breathwork,guided meditation,yoga nidra
date,,,,,
2020-07-26,90,22,1,6,3
2020-08-02,89,21,1,6,3
2020-08-09,90,20,1,6,2
2020-08-16,92,24,1,6,3
2020-08-23,91,23,1,5,3


In [6]:
# Initialize list to store each keyword's region data
country_frames = []

for kw in keywords:
    pytrends.build_payload([kw], timeframe="today 5-y", geo="")
    df_region = pytrends.interest_by_region()

    # Filter out rows with 0 interest
    df_region = df_region[df_region[kw] > 0].reset_index()

    # Rename columns for consistency
    df_region = df_region[["geoName", kw]]
    df_region.columns = ["country", "interest"]
    df_region["keyword"] = kw

    country_frames.append(df_region)

# Combine all keywords into one DataFrame
df_country = pd.concat(country_frames, ignore_index=True)

# Preview result
df_country.head()

,country,interest,keyword
0,Argentina,2,meditation
1,Australia,98,meditation
2,Austria,49,meditation
3,Bangladesh,10,meditation
4,Belgium,25,meditation


In [7]:
# Prepare empty list to collect rows
related_rows = []

# Loop through each keyword
for kw in keywords:
    pytrends.build_payload([kw], timeframe="today 5-y", geo="")

    related = pytrends.related_queries()
    
    if kw in related:
        for qtype in ['top', 'rising']:
            df_q = related[kw].get(qtype)
            if df_q is not None:
                for _, row in df_q.iterrows():
                    related_rows.append({
                        'keyword': kw,
                        'query': row['query'],
                        'type': qtype,
                        'value': row['value']
                    })

# Create combined DataFrame
df_related_queries = pd.DataFrame(related_rows)

# Preview
df_related_queries.head()

,keyword,query,type,value
0,meditation,sleep meditation,top,100
1,meditation,meditation music,top,99
2,meditation,guided meditation,top,74
3,meditation,meditation youtube,top,73
4,meditation,yoga,top,55


In [8]:
os.makedirs("../data/raw", exist_ok=True)

# Save the datasets
df_trends.to_csv("../data/raw/interest_over_time.csv")
df_country.to_csv("../data/raw/interest_by_country.csv")
df_related_queries.to_csv("../data/raw/related_queries.csv")